In [1]:
import numpy as np
import os
import shutil
import subprocess
import pandas as pd


In [2]:


%%file node_info.csv
node_id,region,ServerOrClientregion,cluster_id,key_file,clientID,ServerID, PublicIp
0, 'europe-west3-c', 0, 0, 'FKA.pem', -1, 0,
1, 'europe-west3-c', 0, 0, 'FKA.pem', -1, 1,
2, 'europe-west3-c', 0, 0, 'FKA.pem', -1, 2,
3, 'europe-west3-c', 0, 0, 'FKA.pem', -1, 3,

4, 'europe-west3-c', 1, -1, 'FKA.pem', 0, -1,
5, 'europe-west3-c', 1, -1, 'FKA.pem', 1, -1,
6, 'europe-west3-c', 1, -1, 'FKA.pem', 2, -1,
7, 'europe-west3-c', 1, -1, 'FKA.pem', 3, -1,


8, 'europe-west3-c', 0, 1,'FKA.pem', -1, 4,
9, 'europe-west3-c', 0, 1,'FKA.pem', -1, 5,
10, 'europe-west3-c', 0, 1,'FKA.pem', -1, 6,
11, 'europe-west3-c', 0, 1,'FKA.pem', -1, 7,

12, 'europe-west3-c', 1, -1, 'FKA.pem', 4, -1,
13, 'europe-west3-c', 1, -1, 'FKA.pem', 5, -1,
14, 'europe-west3-c', 1, -1, 'FKA.pem', 6, -1,
15, 'europe-west3-c', 1, -1, 'FKA.pem', 7, -1,



Overwriting node_info.csv


In [3]:


# %%file node_info.csv
# node_id,region,ServerOrClientregion,cluster_id,key_file,clientID,ServerID, PublicIp
# 0, 'europe-west3-c', 0, 0, 'FKA.pem', -1, 0,
# 1, 'europe-west3-c', 0, 0, 'FKA.pem', -1, 1,
# 2, 'europe-west3-c', 0, 0, 'FKA.pem', -1, 2,
# 3, 'europe-west3-c', 0, 0, 'FKA.pem', -1, 3,

# 4, 'europe-west3-c', 1, -1, 'FKA.pem', 0, -1,

# 5, 'europe-west3-c', 0, 1,'FKA.pem', -1, 4,
# 6, 'europe-west3-c', 0, 1,'FKA.pem', -1, 5,
# 7, 'europe-west3-c', 0, 1,'FKA.pem', -1, 6,
# 8, 'europe-west3-c', 0, 1,'FKA.pem', -1, 7,

# 9, 'europe-west3-c', 1, -1, 'FKA.pem', 1, -1,



In [4]:


# %%file node_info.csv
# node_id,region,ServerOrClientregion,cluster_id,key_file,clientID,ServerID, PublicIp
# 0, 'europe-west3-c', 0, 0, 'FKA.pem', -1, 0,
# 1, 'europe-west3-c', 0, 0, 'FKA.pem', -1, 1,
# 2, 'europe-west3-c', 0, 0, 'FKA.pem', -1, 2,
# 3, 'europe-west3-c', 0, 0, 'FKA.pem', -1, 3,

# 4, 'europe-west3-c', 1, -1, 'FKA.pem', 0, -1,

# 5, 'asia-south1-c', 0, 1,'FKA.pem', -1, 4,
# 6, 'asia-south1-c', 0, 1,'FKA.pem', -1, 5,
# 7, 'asia-south1-c', 0, 1,'FKA.pem', -1, 6,
# 8, 'asia-south1-c', 0, 1,'FKA.pem', -1, 7,

# 9, 'asia-south1-c', 1, -1, 'FKA.pem', 1, -1,







In [5]:
df =pd.read_csv('node_info.csv')

node_info_dict = df.to_dict()

node_info_dict

{'node_id': {0: 0,
  1: 1,
  2: 2,
  3: 3,
  4: 4,
  5: 5,
  6: 6,
  7: 7,
  8: 8,
  9: 9,
  10: 10,
  11: 11,
  12: 12,
  13: 13,
  14: 14,
  15: 15},
 'region': {0: " 'europe-west3-c'",
  1: " 'europe-west3-c'",
  2: " 'europe-west3-c'",
  3: " 'europe-west3-c'",
  4: " 'europe-west3-c'",
  5: " 'europe-west3-c'",
  6: " 'europe-west3-c'",
  7: " 'europe-west3-c'",
  8: " 'europe-west3-c'",
  9: " 'europe-west3-c'",
  10: " 'europe-west3-c'",
  11: " 'europe-west3-c'",
  12: " 'europe-west3-c'",
  13: " 'europe-west3-c'",
  14: " 'europe-west3-c'",
  15: " 'europe-west3-c'"},
 'ServerOrClientregion': {0: 0,
  1: 0,
  2: 0,
  3: 0,
  4: 1,
  5: 1,
  6: 1,
  7: 1,
  8: 0,
  9: 0,
  10: 0,
  11: 0,
  12: 1,
  13: 1,
  14: 1,
  15: 1},
 'cluster_id': {0: 0,
  1: 0,
  2: 0,
  3: 0,
  4: -1,
  5: -1,
  6: -1,
  7: -1,
  8: 1,
  9: 1,
  10: 1,
  11: 1,
  12: -1,
  13: -1,
  14: -1,
  15: -1},
 'key_file': {0: " 'FKA.pem'",
  1: " 'FKA.pem'",
  2: " 'FKA.pem'",
  3: " 'FKA.pem'",
  4: " 'FKA

In [6]:
total_count = 0

for key in node_info_dict['ServerOrClientregion'].keys():
    if node_info_dict['ServerOrClientregion'][key] ==0:
        total_count = total_count + 1
        
total_count

8

In [7]:
replica_count = 0

for key in node_info_dict['cluster_id'].keys():
    if node_info_dict['cluster_id'][key] ==0:
        replica_count = replica_count + 1
        
        
replica_count       

4

In [8]:
node_info_dict

{'node_id': {0: 0,
  1: 1,
  2: 2,
  3: 3,
  4: 4,
  5: 5,
  6: 6,
  7: 7,
  8: 8,
  9: 9,
  10: 10,
  11: 11,
  12: 12,
  13: 13,
  14: 14,
  15: 15},
 'region': {0: " 'europe-west3-c'",
  1: " 'europe-west3-c'",
  2: " 'europe-west3-c'",
  3: " 'europe-west3-c'",
  4: " 'europe-west3-c'",
  5: " 'europe-west3-c'",
  6: " 'europe-west3-c'",
  7: " 'europe-west3-c'",
  8: " 'europe-west3-c'",
  9: " 'europe-west3-c'",
  10: " 'europe-west3-c'",
  11: " 'europe-west3-c'",
  12: " 'europe-west3-c'",
  13: " 'europe-west3-c'",
  14: " 'europe-west3-c'",
  15: " 'europe-west3-c'"},
 'ServerOrClientregion': {0: 0,
  1: 0,
  2: 0,
  3: 0,
  4: 1,
  5: 1,
  6: 1,
  7: 1,
  8: 0,
  9: 0,
  10: 0,
  11: 0,
  12: 1,
  13: 1,
  14: 1,
  15: 1},
 'cluster_id': {0: 0,
  1: 0,
  2: 0,
  3: 0,
  4: -1,
  5: -1,
  6: -1,
  7: -1,
  8: 1,
  9: 1,
  10: 1,
  11: 1,
  12: -1,
  13: -1,
  14: -1,
  15: -1},
 'key_file': {0: " 'FKA.pem'",
  1: " 'FKA.pem'",
  2: " 'FKA.pem'",
  3: " 'FKA.pem'",
  4: " 'FKA

In [9]:
n_clusters = int(total_count / replica_count)

nclients = 0

for i in node_info_dict['ServerOrClientregion']:
#     print(node_info_dict['ServerOrClientregion'][i])
    if node_info_dict['ServerOrClientregion'][i] == 1:
        nclients = nclients + 1
nclients

8

In [10]:
clusters_tracked = []
cluster_regions_tracked = []

for key in node_info_dict['region']:
    if int(node_info_dict['cluster_id'][key]) not in clusters_tracked and int(node_info_dict['cluster_id'][key]) >=0:
        clusters_tracked.append(int(node_info_dict['cluster_id'][key]))
        cluster_regions_tracked.append(node_info_dict['region'][key].strip().replace('"', '')[1:-1])
        
        
cluster_regions_tracked     

['europe-west3-c', 'europe-west3-c']

In [11]:
cluster_regions_tracked, clusters_tracked

(['europe-west3-c', 'europe-west3-c'], [0, 1])

In [12]:

regions = cluster_regions_tracked


In [13]:
# regions = [ 'us-west1-b','us-west1-b', 'us-west1-b','us-west1-b',\
#            'europe-west3-c', 'europe-west3-c', 'europe-west3-c', 'europe-west3-c', \
#            'asia-south1-c', 'asia-south1-c', 'asia-south1-c', 'asia-south1-c']


# regions = [ 'us-west1-b','us-west1-b', 'us-west1-b','us-west1-b']


# regions = [ 'us-west1-b','us-west1-b', 'us-west1-b','us-west1-b',\
#            'us-west1-b', 'us-west1-b', 'us-west1-b', 'us-west1-b', \
#            'us-west1-b', 'us-west1-b', 'us-west1-b', 'us-west1-b']



# regions = [ 'europe-west3-c', 'asia-south1-c']

# regions = [ 'us-west1-b', 'us-west1-b']



# regions = [ 'us-west1-b', 'us-west1-b','us-west1-b','us-west1-b', 'us-west1-b', 'us-west1-b','us-west1-b','us-west1-b', 'us-west1-b', 'us-west1-b','us-west1-b','us-west1-b']
# regions = [ 'us-west1-b', 'us-west1-b', 'us-west1-b', 'us-west1-b']



# regions = ['europe-west3-c',  \
#            'asia-south1-c']


n_regions = len(regions)

In [14]:
for i,k in enumerate(regions):
    print(i,k)

0 europe-west3-c
1 europe-west3-c


In [15]:
for item, idx in enumerate(node_info_dict['region'].keys()):
    print(item, idx, node_info_dict['region'][item])

0 0  'europe-west3-c'
1 1  'europe-west3-c'
2 2  'europe-west3-c'
3 3  'europe-west3-c'
4 4  'europe-west3-c'
5 5  'europe-west3-c'
6 6  'europe-west3-c'
7 7  'europe-west3-c'
8 8  'europe-west3-c'
9 9  'europe-west3-c'
10 10  'europe-west3-c'
11 11  'europe-west3-c'
12 12  'europe-west3-c'
13 13  'europe-west3-c'
14 14  'europe-west3-c'
15 15  'europe-west3-c'


In [16]:
lines  = []
# zones = {}


# lim = 0
# for reg in regions:
#     for i in range(replica_count+int(nclients/n_clusters)):
#         line = 'gcloud compute instances create resdb-'+f"{lim+i:02}"+' --project=ucr-ursa-major-lesani-lab --zone='+reg+' --machine-type=e2-highmem-2 --network-interface=network-tier=PREMIUM,stack-type=IPV4_ONLY,subnet=default --can-ip-forward --maintenance-policy=MIGRATE --provisioning-model=STANDARD --service-account=961693926925-compute@developer.gserviceaccount.com --scopes=https://www.googleapis.com/auth/devstorage.read_only,https://www.googleapis.com/auth/logging.write,https://www.googleapis.com/auth/monitoring.write,https://www.googleapis.com/auth/servicecontrol,https://www.googleapis.com/auth/service.management.readonly,https://www.googleapis.com/auth/trace.append --tags=http-server,https-server --create-disk=auto-delete=yes,boot=yes,device-name=instance-3,image=projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20230831,mode=rw,size=10,type=projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/diskTypes/pd-balanced --no-shielded-secure-boot --shielded-vtpm --shielded-integrity-monitoring --labels=goog-ec-src=vm_add-gcloud --reservation-affinity=any'
        
#         print(line)
#         lines.append(line)
#         zones[i] = reg
    
#     lim = lim + replica_count+int(nclients/n_clusters)
        
    
    
    
# with open('launch_cloud_instances.sh', 'w') as f:
#     for item in lines:
#         f.write(str(item) + '\n')


for idx, key in enumerate( node_info_dict['region'].keys()):

    line = 'gcloud compute instances create resdb-'+f"{idx:02}"+' --project=ucr-ursa-major-lesani-lab --zone='+node_info_dict['region'][key].strip()[1:-1]+' --machine-type=e2-highmem-2 --network-interface=network-tier=PREMIUM,stack-type=IPV4_ONLY,subnet=default --can-ip-forward --maintenance-policy=MIGRATE --provisioning-model=STANDARD --service-account=961693926925-compute@developer.gserviceaccount.com --scopes=https://www.googleapis.com/auth/devstorage.read_only,https://www.googleapis.com/auth/logging.write,https://www.googleapis.com/auth/monitoring.write,https://www.googleapis.com/auth/servicecontrol,https://www.googleapis.com/auth/service.management.readonly,https://www.googleapis.com/auth/trace.append --tags=http-server,https-server --create-disk=auto-delete=yes,boot=yes,device-name=instance-3,image=projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20230831,mode=rw,size=10,type=projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/diskTypes/pd-balanced --no-shielded-secure-boot --shielded-vtpm --shielded-integrity-monitoring --labels=goog-ec-src=vm_add-gcloud --reservation-affinity=any'
    lines.append(line)
    
with open('launch_cloud_instances.sh', 'w') as f:
    for item in lines:
        f.write(str(item) + '\n')


In [17]:
len(lines), node_info_dict['region'][key].strip()[1:-1]

(16, 'europe-west3-c')

In [18]:
import subprocess
import concurrent.futures

def run_command(command):
    subprocess.call(command, shell=True)

commands = lines

# Create a ThreadPoolExecutor or ProcessPoolExecutor, depending on your needs
with concurrent.futures.ThreadPoolExecutor(max_workers=len(commands)) as executor:
    # Submit each command to the executor for parallel execution
    futures = [executor.submit(run_command, command) for command in commands]

    # Wait for all tasks to complete
    concurrent.futures.wait(futures)


ERROR: (gcloud.compute.instances.create) Could not fetch resource:
 - The resource 'projects/ucr-ursa-major-lesani-lab/zones/europe-west3-c/instances/resdb-10' already exists

ERROR: (gcloud.compute.instances.create) Could not fetch resource:
 - The resource 'projects/ucr-ursa-major-lesani-lab/zones/europe-west3-c/instances/resdb-03' already exists

ERROR: (gcloud.compute.instances.create) Could not fetch resource:
 - The resource 'projects/ucr-ursa-major-lesani-lab/zones/europe-west3-c/instances/resdb-08' already exists

ERROR: (gcloud.compute.instances.create) Could not fetch resource:
 - The resource 'projects/ucr-ursa-major-lesani-lab/zones/europe-west3-c/instances/resdb-00' already exists

ERROR: (gcloud.compute.instances.create) Could not fetch resource:
 - The resource 'projects/ucr-ursa-major-lesani-lab/zones/europe-west3-c/instances/resdb-09' already exists

ERROR: (gcloud.compute.instances.create) Could not fetch resource:
 - The resource 'projects/ucr-ursa-major-lesani-lab/z

In [19]:
# os.system('sh launch_cloud_instances.sh')

In [20]:
for folder in os.listdir('.'):
    if 'config' in folder and folder!='config' and 'zip' not in folder and '.' not in folder:
        shutil.rmtree(folder)

In [21]:
# os.system("aws --region ap-south-1 ec2 describe-instances --filter Name=instance-type,Values=t2.small --query 'Reservations[*].Instances[*].[PrivateIpAddress]' --output text > all_internal_ips")

# os.system("aws --region ap-south-1 ec2 describe-instances  --filter Name=instance-type,Values=t2.small --query 'Reservations[*].Instances[*].[PublicIpAddress]' --output text > all_external_ips")


instance_type = 'e2-small'


redirect = '>'

os.system('gcloud compute instances list  --format="value(networkInterfaces[0].networkIP)" '+\
          redirect+' all_internal_ips')



os.system('gcloud compute instances list --format="value(networkInterfaces[0].networkIP)" '+\
      redirect+' all_external_ips')












0

In [22]:
with open('all_internal_ips') as f:
    ipList = f.read().splitlines()
ipList = [x for x in ipList if x!='None']

In [23]:
ipList, len(ipList)

(['10.156.0.65',
  '10.156.0.64',
  '10.156.0.67',
  '10.156.0.66',
  '10.156.0.68',
  '10.156.0.59',
  '10.156.0.62',
  '10.156.0.63',
  '10.156.0.60',
  '10.156.0.61',
  '10.156.0.73',
  '10.156.0.69',
  '10.156.0.71',
  '10.156.0.74',
  '10.156.0.70',
  '10.156.0.72'],
 16)

In [24]:
new_server_only_ips= []
new_client_only_ips= []



for i in range(total_count + nclients):
    if node_info_dict['ServerOrClientregion'][i] ==0:
        new_server_only_ips = new_server_only_ips + [ipList[i]]
    else:
        new_client_only_ips = new_client_only_ips + [ipList[i]]
        
        

In [25]:
# new_server_only_ips= []
# new_client_only_ips= []


# current_region_total = 0
# for i in range(n_regions):
#     print(current_region_total,current_region_total+ clusters_per_region_map[i] *replica_count)
#     new_server_only_ips = new_server_only_ips + ipList[current_region_total:current_region_total+ clusters_per_region_map[i] *replica_count]
    
#     new_client_only_ips = new_client_only_ips + ipList[current_region_total+ clusters_per_region_map[i] *replica_count:
#                                                       current_region_total+ clusters_per_region_map[i] *(replica_count+1)]
    
    
#     current_region_total = current_region_total + clusters_per_region_map[i] *replica_count +  clusters_per_region_map[i]
    
    

In [26]:
new_server_only_ips, len(new_server_only_ips)

(['10.156.0.65',
  '10.156.0.64',
  '10.156.0.67',
  '10.156.0.66',
  '10.156.0.60',
  '10.156.0.61',
  '10.156.0.73',
  '10.156.0.69'],
 8)

In [27]:
new_client_only_ips, len(new_client_only_ips) 

(['10.156.0.68',
  '10.156.0.59',
  '10.156.0.62',
  '10.156.0.63',
  '10.156.0.71',
  '10.156.0.74',
  '10.156.0.70',
  '10.156.0.72'],
 8)

In [28]:
ipList = new_server_only_ips + new_client_only_ips

In [29]:
ipList, len(ipList)

(['10.156.0.65',
  '10.156.0.64',
  '10.156.0.67',
  '10.156.0.66',
  '10.156.0.60',
  '10.156.0.61',
  '10.156.0.73',
  '10.156.0.69',
  '10.156.0.68',
  '10.156.0.59',
  '10.156.0.62',
  '10.156.0.63',
  '10.156.0.71',
  '10.156.0.74',
  '10.156.0.70',
  '10.156.0.72'],
 16)

In [30]:
nclients = 0
nservers = 0
count = 0

clusterIpMap = {}


for i in node_info_dict['ServerOrClientregion']:
#     print(node_info_dict['ServerOrClientregion'][i])

    if node_info_dict['ServerOrClientregion'][i] == 0:
        clusterIpMap[nservers] = ipList[count]
        nservers = nservers + 1
        
    count = count + 1
    
nclients = 0
nservers = 0
count = 0

for i in node_info_dict['ServerOrClientregion']:
#     print(node_info_dict['ServerOrClientregion'][i])

    if node_info_dict['ServerOrClientregion'][i] == 1:
        clusterIpMap[7001+nclients] = ipList[count]
        nclients = nclients + 1
        
    count = count + 1
    


In [31]:
clusterIpMap

{0: '10.156.0.65',
 1: '10.156.0.64',
 2: '10.156.0.67',
 3: '10.156.0.66',
 4: '10.156.0.68',
 5: '10.156.0.59',
 6: '10.156.0.62',
 7: '10.156.0.63',
 7001: '10.156.0.60',
 7002: '10.156.0.61',
 7003: '10.156.0.73',
 7004: '10.156.0.69',
 7005: '10.156.0.71',
 7006: '10.156.0.74',
 7007: '10.156.0.70',
 7008: '10.156.0.72'}

In [32]:
# clusterIpMap = {}

# for i in range(len(ipList)):
#     clusterIpMap[i] = ipList[i]

    
# for i in range(nclients):
    
#     clusterIpMap[7001+i] = ipList[-(nclients-i)]

In [33]:
clusterPortMap1 = {}
clusterPortMap2 = {}

for i in range(len(ipList) - nclients):
    
    clusterPortMap1[i] = str(10000)
    
for i in range(len(ipList) - nclients):
    clusterPortMap2[i] = str(20000)
    
# clusterPortMap1[7003] = '11400'
# clusterPortMap2[7003] = '11410'



temp = 11900

for i in range(nclients):
    
    clusterPortMap1[7001+i] = str(10000)
    clusterPortMap2[7001+i] = str(20000)

In [34]:
clusterPortMap1, clusterPortMap2

({0: '10000',
  1: '10000',
  2: '10000',
  3: '10000',
  4: '10000',
  5: '10000',
  6: '10000',
  7: '10000',
  7001: '10000',
  7002: '10000',
  7003: '10000',
  7004: '10000',
  7005: '10000',
  7006: '10000',
  7007: '10000',
  7008: '10000'},
 {0: '20000',
  1: '20000',
  2: '20000',
  3: '20000',
  4: '20000',
  5: '20000',
  6: '20000',
  7: '20000',
  7001: '20000',
  7002: '20000',
  7003: '20000',
  7004: '20000',
  7005: '20000',
  7006: '20000',
  7007: '20000',
  7008: '20000'})

In [35]:
os.system('echo 3108 | sudo -S ./gradlew installDist')

[sudo] password for tejas: 


> Task :compileJava

> Task :processResources NO-SOURCE
> Task :classes
> Task :jar
> Task :installDist

BUILD SUCCESSFUL in 1s
3 actionable tasks: 3 executed


Note: /home/tejas/work/BftSmart_Final/src/main/java/bftsmart/reconfiguration/ViewController.java uses or overrides a deprecated API.
Note: Recompile with -Xlint:deprecation for details.
Note: Some input files use unchecked or unsafe operations.
Note: Recompile with -Xlint:unchecked for details.


0

In [36]:
for i in range(total_count):
    os.system('./runscripts/smartrun.sh bftsmart.tom.util.RSAKeyPairGenerator '+str(i)+' 1024')

    

os.system('./runscripts/smartrun.sh bftsmart.tom.util.RSAKeyPairGenerator '+str(7002)+' 1024')
os.system('./runscripts/smartrun.sh bftsmart.tom.util.RSAKeyPairGenerator '+str(7001)+' 1024')

0

In [37]:
os.system('echo 3108 | sudo -S ./gradlew clean')

[sudo] password for tejas: 

> Task :clean

BUILD SUCCESSFUL in 1s
1 actionable task: 1 executed


0

In [38]:
for key in clusterPortMap1.keys():
    if key>7002:
        print(key)
        print()
        
        for folder in os.listdir('config/'):
            if os.path.isdir('config/'+folder) and 'keys' in folder:
                if os.path.exists('config/'+folder+'/'+'privatekey7001'):
                    shutil.copy('config/'+folder+'/'+'privatekey7001','config/'+folder+'/'+'privatekey'+str(key))
                    shutil.copy('config/'+folder+'/'+'publickey7001','config/'+folder+'/'+'publickey'+str(key))
                
        print('----------')
        print()

7003

----------

7004

----------

7005

----------

7006

----------

7007

----------

7008

----------



In [39]:
for i in range(n_clusters):
    
    if os.path.exists('config'+str(i)):
        shutil.rmtree('config'+str(i))
    
    os.mkdir('config'+str(i))
    os.mkdir('config'+str(i)+'/keysRSA')
    os.mkdir('config'+str(i)+'/keysSunEC')
    os.mkdir('config'+str(i)+'/keysECDSA')
    
    print('config'+str(i)+'/keysECDSA')
    
    
    shutil.copytree('config/workloads/', 'config'+str(i)+'/workloads')
    shutil.copytree('config/keysSSL_TLS/', 'config'+str(i)+'/keysSSL_TLS')
    print('config/workloads/', 'config'+str(i)+'/workloads')

    
    for file in os.listdir('config/keys/'):
    
        shutil.copy('config/keys/'+file, 'config'+str(i)+'/keysRSA/'+file)
        shutil.copy('config/keys/'+file, 'config'+str(i)+'/keysSunEC/'+file)
        shutil.copy('config/keys/'+file, 'config'+str(i)+'/keysECDSA/'+file)
    
    
    shutil.copy('config/system.config', 'config'+str(i)+'/system.config')    
    shutil.copy('config/logback.xml', 'config'+str(i)+'/logback.xml')
    
    
    
    

config0/keysECDSA
config/workloads/ config0/workloads
config1/keysECDSA
config/workloads/ config1/workloads


In [40]:
        
        for i in range(nclients):
            print(str(7001+i)+' '+clusterIpMap[7001+i]+' '+clusterPortMap1[7001+i]+'\n')

7001 10.156.0.60 10000

7002 10.156.0.61 10000

7003 10.156.0.73 10000

7004 10.156.0.69 10000

7005 10.156.0.71 10000

7006 10.156.0.74 10000

7007 10.156.0.70 10000

7008 10.156.0.72 10000



In [41]:
clusterIpMap, clusterPortMap1

({0: '10.156.0.65',
  1: '10.156.0.64',
  2: '10.156.0.67',
  3: '10.156.0.66',
  4: '10.156.0.68',
  5: '10.156.0.59',
  6: '10.156.0.62',
  7: '10.156.0.63',
  7001: '10.156.0.60',
  7002: '10.156.0.61',
  7003: '10.156.0.73',
  7004: '10.156.0.69',
  7005: '10.156.0.71',
  7006: '10.156.0.74',
  7007: '10.156.0.70',
  7008: '10.156.0.72'},
 {0: '10000',
  1: '10000',
  2: '10000',
  3: '10000',
  4: '10000',
  5: '10000',
  6: '10000',
  7: '10000',
  7001: '10000',
  7002: '10000',
  7003: '10000',
  7004: '10000',
  7005: '10000',
  7006: '10000',
  7007: '10000',
  7008: '10000'})

In [42]:
def find_key_by_value(dictionary, value):
    for key, val in dictionary.items():
        if val == value:
            return key
    # If the value is not found, you might want to handle this case accordingly.
    return None


In [43]:
node_info_dict

{'node_id': {0: 0,
  1: 1,
  2: 2,
  3: 3,
  4: 4,
  5: 5,
  6: 6,
  7: 7,
  8: 8,
  9: 9,
  10: 10,
  11: 11,
  12: 12,
  13: 13,
  14: 14,
  15: 15},
 'region': {0: " 'europe-west3-c'",
  1: " 'europe-west3-c'",
  2: " 'europe-west3-c'",
  3: " 'europe-west3-c'",
  4: " 'europe-west3-c'",
  5: " 'europe-west3-c'",
  6: " 'europe-west3-c'",
  7: " 'europe-west3-c'",
  8: " 'europe-west3-c'",
  9: " 'europe-west3-c'",
  10: " 'europe-west3-c'",
  11: " 'europe-west3-c'",
  12: " 'europe-west3-c'",
  13: " 'europe-west3-c'",
  14: " 'europe-west3-c'",
  15: " 'europe-west3-c'"},
 'ServerOrClientregion': {0: 0,
  1: 0,
  2: 0,
  3: 0,
  4: 1,
  5: 1,
  6: 1,
  7: 1,
  8: 0,
  9: 0,
  10: 0,
  11: 0,
  12: 1,
  13: 1,
  14: 1,
  15: 1},
 'cluster_id': {0: 0,
  1: 0,
  2: 0,
  3: 0,
  4: -1,
  5: -1,
  6: -1,
  7: -1,
  8: 1,
  9: 1,
  10: 1,
  11: 1,
  12: -1,
  13: -1,
  14: -1,
  15: -1},
 'key_file': {0: " 'FKA.pem'",
  1: " 'FKA.pem'",
  2: " 'FKA.pem'",
  3: " 'FKA.pem'",
  4: " 'FKA

In [44]:
find_key_by_value(node_info_dict['ServerID'], 4)

8

In [45]:
for n_cluster in range(n_clusters):
    
    with open('config'+str(n_cluster)+'/hosts.config','w') as file:
        
        for key in clusterIpMap:
            if key < 7000:
                file.writelines(str(key)+' '+clusterIpMap[key]+' '  +clusterPortMap1[key]+' ' +  clusterPortMap2[key]+' '+str(node_info_dict['cluster_id'][find_key_by_value(node_info_dict['ServerID'], key)] )+'\n')

        for key in clusterIpMap:
            if key >= 7000:
                file.writelines(str(key)+' '+clusterIpMap[key]+' '+clusterPortMap1[key]+'\n')

        
        file.close()


In [46]:
# for n_cluster in range(n_clusters):
    
#     with open('config'+str(n_cluster)+'/hosts.config','w') as file:
        
#         for n_cluster1 in range(n_clusters):
        
#             for i_replica in range(replica_count):
#     #             file.writelines(str(i_replica)+' '+clusterIpMap[n_cluster*replica_count+i_replica]+' '  +clusterPortMap1[n_cluster*replica_count+i_replica]+' ' +  clusterPortMap2[n_cluster*replica_count+i_replica]+'\n')
#                 file.writelines(str(n_cluster1*replica_count+i_replica)+' '+clusterIpMap[n_cluster1*replica_count+i_replica]+' '  +clusterPortMap1[n_cluster1*replica_count+i_replica]+' ' +  clusterPortMap2[n_cluster1*replica_count+i_replica]+' '+str(n_cluster1)+'\n')

#         for iter_ in range(len(ipList) - (n_clusters*replica_count)-nclients):
#             print((n_clusters*replica_count) + iter_)
            
# #             file.writelines(str((n_clusters*replica_count) + iter_)+' '+clusterIpMap[(n_clusters*replica_count) + iter_]+' '  +clusterPortMap1[(n_clusters*replica_count) + iter_]+' ' +  clusterPortMap2[(n_clusters*replica_count) + iter_]+' '+str(iter_%n_clusters)+'\n')
#             file.writelines(str((n_clusters*replica_count) + iter_)+' '+clusterIpMap[(n_clusters*replica_count) + iter_]+' '  +clusterPortMap1[(n_clusters*replica_count) + iter_]+' ' +  clusterPortMap2[(n_clusters*replica_count) + iter_]+' '+str(0%n_clusters)+'\n')
        
#         for i in range(nclients):
#             file.writelines(str(7001+i)+' '+clusterIpMap[7001+i]+' '+clusterPortMap1[7001+i]+'\n')

        
#         file.close()


In [47]:
replica_count

4

In [ ]:
# for n_cluster in range(n_clusters):
#     with open('config'+str(n_cluster)+'/system.config','r') as file:
#         data = file.readlines()
#         f = int(np.ceil((replica_count-1)/3))
#         print('f is',f )
        
#         for n_line in range(len(data)):
#             if ('system.servers.num' in data[n_line]) and '#' not in data[n_line]:
                
                
#                 data[n_line] = 'system.servers.num = '+str(replica_count)
#                 print(data[n_line])
                
#             if ('system.servers.f' in data[n_line]) and '#' not in data[n_line]:
#                 data[n_line] = 'system.servers.f = '+str(f)
#                 print(data[n_line])
                
                
#             if ('system.initial.view' in data[n_line]) and '#' not in data[n_line]:
#                 data[n_line] = 'system.initial.view = '
#                 for iter_view in range(replica_count):
#                     data[n_line] = data[n_line] +str(n_cluster*replica_count+iter_view)+','
# #                     data[n_line] = data[n_line] +str(n_cluster*replica_count+iter_view)+','
#                     print(data[n_line])
                
#                 data[n_line] = data[n_line][:-1]
#                 print(data[n_line])
        
                
                
#         file.close()
        
#     with open('config'+str(n_cluster)+'/system.config', 'w') as f:
#         for line in data:
#             f.write(f"{line}\n")
        

In [59]:
for n_cluster in range(n_clusters):
    with open('config'+str(n_cluster)+'/system.config','r') as file:
        data = file.readlines()

        
        nservers = 0
        server_id_list = []



        for i in node_info_dict['ServerOrClientregion']:
            if node_info_dict['ServerOrClientregion'][i] == 0 and \
            node_info_dict['cluster_id'][i]  == n_cluster:
                nservers = nservers + 1
                server_id_list.append(node_info_dict['ServerID'][i])

        f = int(np.ceil((nservers-1)/3))
        print('f is',f )
        
        
        for n_line in range(len(data)):
            if ('system.servers.num' in data[n_line]) and '#' not in data[n_line]:
                
                
                data[n_line] = 'system.servers.num = '+str(nservers)
                print(data[n_line])
                
            if ('system.servers.f' in data[n_line]) and '#' not in data[n_line]:
                data[n_line] = 'system.servers.f = '+str(f)
                print(data[n_line])
                
                
            if ('system.initial.view' in data[n_line]) and '#' not in data[n_line]:
                data[n_line] = 'system.initial.view = '
                for iter_view in server_id_list:
                    data[n_line] = data[n_line] +str(iter_view)+','
#                     data[n_line] = data[n_line] +str(n_cluster*replica_count+iter_view)+','
                    print(data[n_line])
                
                data[n_line] = data[n_line][:-1]
                print(data[n_line])
        
                
                
        file.close()
        
    with open('config'+str(n_cluster)+'/system.config', 'w') as f:
        for line in data:
            f.write(f"{line}\n")
        

f is 1
system.servers.num = 4
system.servers.f = 1
system.initial.view = 0,
system.initial.view = 0,1,
system.initial.view = 0,1,2,
system.initial.view = 0,1,2,3,
system.initial.view = 0,1,2,3
f is 1
system.servers.num = 4
system.servers.f = 1
system.initial.view = 4,
system.initial.view = 4,5,
system.initial.view = 4,5,6,
system.initial.view = 4,5,6,7,
system.initial.view = 4,5,6,7


In [60]:
for n_cluster in range(n_clusters):
    for i in range(400):
        
        for ncls in  range(1):
        
        
            print('config'+str(n_cluster)+'/keysRSA/'+'publickey7002', \
                        'config'+str(n_cluster)+'/keysRSA/'+'publickey'+str(1000+(ncls*100)+i))
            shutil.copy('config'+str(n_cluster)+'/keysRSA/'+'publickey7002', \
                        'config'+str(n_cluster)+'/keysRSA/'+'publickey'+str(1000+(ncls*100)+i))
            shutil.copy('config'+str(n_cluster)+'/keysRSA/'+'privatekey7002', \
                        'config'+str(n_cluster)+'/keysRSA/'+'privatekey'+str(1000+(ncls*100)+i))
        
#         shutil.copy('config'+str(n_cluster)+'/keysRSA/'+'publickey7002', \
#                     'config'+str(n_cluster)+'/keysRSA/'+'publickey7002')
        
#         shutil.copy('config'+str(n_cluster)+'/keysRSA/'+'privatekey7002', \
#                     'config'+str(n_cluster)+'/keysRSA/'+'privatekey7002')

config0/keysRSA/publickey7002 config0/keysRSA/publickey1000
config0/keysRSA/publickey7002 config0/keysRSA/publickey1001
config0/keysRSA/publickey7002 config0/keysRSA/publickey1002
config0/keysRSA/publickey7002 config0/keysRSA/publickey1003
config0/keysRSA/publickey7002 config0/keysRSA/publickey1004
config0/keysRSA/publickey7002 config0/keysRSA/publickey1005
config0/keysRSA/publickey7002 config0/keysRSA/publickey1006
config0/keysRSA/publickey7002 config0/keysRSA/publickey1007
config0/keysRSA/publickey7002 config0/keysRSA/publickey1008
config0/keysRSA/publickey7002 config0/keysRSA/publickey1009
config0/keysRSA/publickey7002 config0/keysRSA/publickey1010
config0/keysRSA/publickey7002 config0/keysRSA/publickey1011
config0/keysRSA/publickey7002 config0/keysRSA/publickey1012
config0/keysRSA/publickey7002 config0/keysRSA/publickey1013
config0/keysRSA/publickey7002 config0/keysRSA/publickey1014
config0/keysRSA/publickey7002 config0/keysRSA/publickey1015
config0/keysRSA/publickey7002 config0/ke

In [61]:
# os.system('git remote set-url origin https://github.com/tmane002/BftSmart_Final.git')

In [62]:
import os
os.system('git add .; git commit -m "Working with modified ClusterInfo";git push')

[main 80b33a6b] Working with modified ClusterInfo
 3 files changed, 59101 insertions(+), 381 deletions(-)


To github.com:tmane002/BftSmart_Final.git
   863f0721..80b33a6b  main -> main


0

In [52]:
# 'gcloud compute instances create instance-1 --project=cellular-hybrid-358402 --zone=us-central1-a --machine-type=e2-medium --network-interface=network-tier=PREMIUM,subnet=default --no-restart-on-failure --maintenance-policy=TERMINATE --provisioning-model=SPOT --instance-termination-action=STOP --service-account=932709639592-compute@developer.gserviceaccount.com --scopes=https://www.googleapis.com/auth/devstorage.read_only,https://www.googleapis.com/auth/logging.write,https://www.googleapis.com/auth/monitoring.write,https://www.googleapis.com/auth/servicecontrol,https://www.googleapis.com/auth/service.management.readonly,https://www.googleapis.com/auth/trace.append --tags=http-server,https-server --create-disk=auto-delete=yes,boot=yes,device-name=instance-1,image=projects/ubuntu-os-cloud/global/images/ubuntu-2204-jammy-v20220810,mode=rw,size=10,type=projects/cellular-hybrid-358402/zones/us-central1-a/diskTypes/pd-balanced --no-shielded-secure-boot --shielded-vtpm --shielded-integrity-monitoring --reservation-affinity=any'

In [53]:
# gcloud compute instances create instance-1 --project=cellular-hybrid-358402 --zone=us-west1-b --machine-type=e2-medium --network-interface=network-tier=PREMIUM,subnet=default --no-restart-on-failure --maintenance-policy=TERMINATE --provisioning-model=SPOT --instance-termination-action=STOP --service-account=932709639592-compute@developer.gserviceaccount.com --scopes=https://www.googleapis.com/auth/devstorage.read_only,https://www.googleapis.com/auth/logging.write,https://www.googleapis.com/auth/monitoring.write,https://www.googleapis.com/auth/servicecontrol,https://www.googleapis.com/auth/service.management.readonly,https://www.googleapis.com/auth/trace.append --tags=http-server,https-server --create-disk=auto-delete=yes,boot=yes,device-name=instance-1,image=projects/ubuntu-os-cloud/global/images/ubuntu-2204-jammy-v20220810,mode=rw,size=10,type=projects/cellular-hybrid-358402/zones/us-central1-a/diskTypes/pd-balanced --no-shielded-secure-boot --shielded-vtpm --shielded-integrity-monitoring --reservation-affinity=any